In [107]:
# %load first_cell.py
%reload_ext autoreload
%autoreload 2
from pathlib import Path
home = str(Path.home())

import sys
sys.path = sys.path + [f'{home}/.conda/envs/norm_env/lib/python37.zip', 
                       f'{home}/.conda/envs/norm_env/lib/python3.7', 
                       f'{home}/.conda/envs/norm_env/lib/python3.7/lib-dynload', 
                       f'{home}/.conda/envs/norm_env/lib/python3.7/site-packages',
                     '../src']
sys.prefix = '/home/joaom/.conda/envs/norm_env'

from paths import RAW_PATH, TREAT_PATH, OUTPUT_PATH, FIGURES_PATH

from copy import deepcopy
import numpy as np
import pandas as pd
pd.options.display.max_columns = 999
import yaml
import matplotlib.pyplot as plt 
import datetime

import warnings
warnings.filterwarnings('ignore')

# Plotting
import plotly
import plotly.graph_objs as go
import cufflinks as cf
plotly.offline.init_notebook_mode(connected=True)

def iplottitle(title, width=40):
    return '<br>'.join(textwrap.wrap(title, width))

# Setting cufflinks
import textwrap
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)
import yaml
from jinja2 import Template
cf.themes.THEMES['custom'] = yaml.load(open('cufflinks_template.yaml', 'r'))

# Imports

In [108]:
sys.path.append('../')
from src import utils
from shapely import wkt
conn = utils.connect_athena(path='../configs/athena.yaml')
import json

In [109]:
metadata = pd.read_sql_query(f"""
select 
    *
from spd_sdv_waze_corona.prod_analysis_metadata_variation
""", conn)

In [52]:
def get_length(j):
    a = pd.DataFrame([r['tags'] for r in j['elements']])
    if len(a):
        return round(a['length'].astype(float).sum(), 2)

paths = (RAW_PATH / 'osm_region_length')
names = [p.name for p in paths.glob('*')]
length = [get_length(json.load(open(s, 'r'))) for s in paths.glob('*')]
length = pd.DataFrame(list(zip(names, length)), columns=['region_slug', 'total_length'])
length['region_slug'] = length['region_slug'].apply(lambda x: '_'.join(x.split('_')[:-1]))
length = length.groupby('region_slug').sum().sort_values(by='total_length').reset_index()

In [53]:
df = metadata.merge(length, on='region_slug', how='outer')

In [110]:
df = metadata

In [111]:
df = df.dropna(subset=['region_shapefile_wkt'])

In [91]:
df.to_csv(OUTPUT_PATH / 'TCI_osmlength.csv', index=False, encoding='latin1')

ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'

In [92]:
df

,country_name,country_iso,region_slug,region_name,region_type,population,dashboard,timezone,region_shapefile_wkt,rerun,osm_length,idb_code,waze_code,country_name_idb_eng,region_shapefile_binary,area,n_days,daily_mean,daily_std,daily_coef_var,weekly_mean,weekly_std,weekly_coef_var,daily_osm_ratio,weekly_osm_ratio,daily_approved,weekly_approved
0,Jamaica,JM,kingston,Kingston,city,585459,TRUE,America/Jamaica,Polygon ((-76.80041645747992618 18.08291869684...,FALSE,1.377157e+06,JA,JM,Jamaica,b'\x00\x00\x00\x00\x03\x05\x00\x00\x00\x9e{B[\...,0.013125,365.0,5.576439e+05,2.739867e+05,0.491329,3.840378e+06,1.035802e+06,0.269713,0.404924,0.398375,True,True
1,Honduras,HN,tegucigalpa,Tegucigalpa,city,564067,TRUE,America/Tegucigalpa,Polygon ((-87.21708244142224942 14.18291843040...,FALSE,2.036947e+06,HO,HO,Honduras,b'\x00\x00\x00\x00\x03\x05\x00\x00\x00\r\xde/%...,0.024583,365.0,2.030470e+06,9.516658e+05,0.468692,1.398342e+07,4.804925e+06,0.343616,0.996820,0.980699,True,True
2,Costa Rica,CR,country_costa_rica,Costa Rica,country,5094114,TRUE,America/Costa_Rica,MULTIPOLYGON (((-85.7025586498513 11.074584884...,FALSE,4.192138e+07,CR,CS,Costa Rica,b'\x00\x00\x00\x00\x06\x05\x00\x00\x00\xf0\xa1...,4.365439,365.0,8.168918e+07,1.785129e+07,0.218527,5.625764e+08,8.850142e+07,0.157314,1.948628,1.917114,True,True
3,The Bahamas,BS,country_the_bahamas,The Bahamas,country,393248,TRUE,America/Nassau,"POLYGON ((-80.700194 23.910715, -80.524446 24....",FALSE,5.977117e+06,BH,BF,Bahamas,b'\x00\x00\x00\x00\x03\x05\x00\x00\x00\xbf\xb6...,26.702328,365.0,2.441384e+05,1.091057e+05,0.446901,1.681331e+06,5.442985e+05,0.323731,0.040846,0.040185,False,False
4,Chile,CL,ch_comunas_lo_prado,LO PRADO,submetro,96249,FALSE,America/Santiago,POLYGON ((-70.70558600981737 -33.4571600757547...,FALSE,1.454056e+05,CH,CI,Chile,b'\x00\x00\x00\x00\x03\x05\x00\x00\x00RH\xb6Vi...,0.000642,365.0,2.686769e+05,1.044837e+05,0.388882,1.850322e+06,5.901058e+05,0.318921,1.847776,1.817893,True,True
5,Chile,CL,ch_comunas_lampa,LAMPA,submetro,102034,FALSE,America/Santiago,POLYGON ((-70.79438009896514 -33.1718882159303...,FALSE,5.838576e+05,CH,CI,Chile,b'\x00\x00\x00\x00\x03\x05\x00\x00\x00;\x15\x9...,0.043495,365.0,1.197822e+06,5.434043e+05,0.453660,8.249151e+06,2.418809e+06,0.293219,2.051565,2.018386,True,True
6,Mexico,MX,torreon,Torreón,city,1098998,TRUE,America/Monterrey,Polygon ((-103.3920813812878805 25.64958588044...,FALSE,4.492773e+06,ME,MX,Mexico,b'\x00\x00\x00\x00\x03\x05\x00\x00\x00\x8ab\xa...,0.026458,365.0,2.957517e+06,1.505250e+06,0.508957,2.036780e+07,9.251570e+06,0.454225,0.658283,0.647637,False,True
7,Brasil,BR,br_states_mato_grosso,Mato Grosso,state,,FALSE,,POLYGON ((-54.89485339361059 -17.6215039881558...,FALSE,1.238181e+08,BR,BR,Brazil,b'\x00\x00\x00\x00\x03\x05\x00\x00\x00C\x81\xa...,75.198337,365.0,2.820081e+07,9.818665e+06,0.348170,1.942131e+08,6.144397e+07,0.316374,0.227760,0.224076,True,True
8,Colombia,CO,medellin,Medellín,city,3550671,TRUE,America/Bogota,Polygon ((-75.54208320661969367 6.382917897523...,FALSE,3.636567e+06,CO,CO,Colombia,b'\x00\x00\x00\x00\x03\x05\x00\x00\x00\xec!\xc...,0.039653,365.0,2.328588e+07,8.197524e+06,0.352038,1.603650e+08,3.831025e+07,0.238894,6.403259,6.299702,True,True
9,El Salvador,SV,sansalvador,San Salvador,city,1610523,TRUE,America/El_Salvador,Polygon ((-89.19208231197771397 13.83291840649...,FALSE,2.680897e+06,ES,ES,El Salvador,b'\x00\x00\x00\x00\x03\x05\x00\x00\x00\xcf+\x1...,0.030347,365.0,1.618139e+07,6.522647e+06,0.403096,1.114379e+08,2.668422e+07,0.239454,6.035814,5.938199,True,True


# CV Distributions

In [57]:
df.query('daily_coef_var < 1').pivot_table(columns='region_type', values='daily_coef_var', index='region_slug').iplot(
    kind='hist', theme='custom',
    title='Distribution of Daily CV',
    xTitle='CV', yTitle='# regions',
)

In [58]:
df.query('weekly_coef_var < 1').pivot_table(columns='region_type', values='weekly_coef_var', index='region_slug').iplot(
    kind='hist', theme='custom',
    title='Distribution of Weekly CV',
    xTitle='CV', yTitle='# regions',
)

In [59]:
import plotly.express as px

px.scatter(df.dropna(subset=['weekly_coef_var', 'daily_coef_var'])\
             .query('weekly_coef_var < 1').query('daily_coef_var < 1'),
            x="weekly_coef_var", y="daily_coef_var", color='region_type',
            hover_name="region_slug", template='plotly_white', trendline='ols').show()

In [60]:
fig = px.scatter(df,
            x="daily_mean", y="daily_coef_var", color='region_type',
            hover_name="region_slug", template='plotly_white', 
            log_x=True
          )
fig.update_xaxes(title_text='Daily TCI Mean (m)')
fig.update_yaxes(title_text='Daily CV')

fig.show()

In [61]:
fig = px.scatter(df,
            x="weekly_mean", y="weekly_coef_var", color='region_type',
            hover_name="region_slug", template='plotly_white', 
            log_x=True
          )
fig.update_xaxes(title_text='Weekly TCI Mean (m)')
fig.update_yaxes(title_text='Weekly CV')

fig.show()

In [62]:
fig.write_html(open(OUTPUT_PATH / 'weeklyTCICV.html', 'w'))

In [63]:
df.head()

,country_name,country_iso,region_slug,region_name,region_type,population,dashboard,timezone,region_shapefile_wkt,rerun,idb_code,waze_code,country_name_idb_eng,region_shapefile_binary,area,n_days,daily_mean,daily_std,daily_coef_var,weekly_mean,weekly_std,weekly_coef_var,daily_approved,weekly_approved,total_length
0,Colombia,CO,bogota,Bogotá,city,8770058.0,True,America/Bogota,Polygon ((-74.04208330493199242 4.932917798462...,False,CO,CO,Colombia,b'\x00\x00\x00\x00\x03\x05\x00\x00\x00\x04\x01...,7.152778e-02,365.0,1.087251e+08,3.117014e+07,0.286688,7.487672e+08,1.654536e+08,0.220968,True,True,8552341.21
1,Nicaragua,NI,managua,Managua,city,941824.0,True,America/Managua,Polygon ((-86.35874916434539728 12.17458495986...,False,NI,NU,Nicaragua,b'\x00\x00\x00\x00\x03\x05\x00\x00\x00-\xd1\xb...,2.986111e-02,365.0,2.400173e+06,8.711462e+05,0.362951,1.652949e+07,3.798560e+06,0.229805,True,True,2239902.08
2,Chile,CL,ch_comunas_renca,RENCA,submetro,147151.0,False,America/Santiago,"POLYGON ((334281.3190000001 6301401.4268, 3342...",False,CH,CI,Chile,"b""\x00\x00\x00\x00\x03\x05\x00\x00\x00\xa0\x89...",2.398348e+07,364.0,0.000000e+00,0.000000e+00,NaN,0.000000e+00,0.000000e+00,NaN,False,False,353222.22
3,Guyana,GY,country_guyana,Guyana,country,786559.0,False,America/Guyana,"MULTIPOLYGON (((-61.40894 5.971249,-61.124543 ...",False,GY,GY,Guyana,b'\x00\x00\x00\x00\x06\x05\x00\x00\x00\\\x03[%...,1.722914e+01,365.0,6.931735e+04,5.828429e+04,0.840833,4.773742e+05,2.072765e+05,0.434201,False,True,6241684.02
4,Jamaica,JM,kingston,Kingston,city,585459.0,True,America/Jamaica,Polygon ((-76.80041645747992618 18.08291869684...,False,JA,JM,Jamaica,b'\x00\x00\x00\x00\x03\x05\x00\x00\x00\x9e{B[\...,1.312500e-02,365.0,5.576394e+05,2.739794e+05,0.491320,3.840347e+06,1.035771e+06,0.269708,True,True,1384098.48


# TCP by Network Length

In [64]:
fig = px.scatter(df,
            y="daily_mean", x="total_length", color='region_type',
            hover_name="region_slug", template='plotly_white', title='Daily TCI Mean by Network Length (m)',
           log_x=True, log_y=True)
fig.update_yaxes(title_text='Daily TCI Mean (m)')
fig.update_xaxes(title_text='Total Network Length (m)')

# fig.add_trace()

fig.show()

In [65]:
fig = px.scatter(df,
            y="daily_mean", x="total_length", color='region_type',
            hover_name="region_slug", template='plotly_white', title='Daily TCI Mean by Network Length (m)',
           log_x=True, log_y=True)
fig.update_yaxes(title_text='Daily TCI Mean (m)')
fig.update_xaxes(title_text='Total Network Length (m)')

# fig.add_trace()

fig.show()

In [67]:
df['daily_length_ratio'] =  df['daily_mean'] / df['total_length']

In [68]:
df[df['daily_length_ratio'] < 200]['daily_length_ratio'].apply(np.log10).iplot(
    kind='hist', theme='custom',
    title='Daily TCP Mean over Network Lenth',
    xTitle='Daily TCP Mean over Network Lenth (log10)', yTitle='# regions'
)

In [69]:
fig = px.scatter(df.dropna(subset=['weekly_coef_var', 'daily_coef_var'])\
             .query('weekly_coef_var < 1').query('daily_coef_var < 1'),
            y="weekly_mean", x="total_length", color='region_type',
            hover_name="region_slug", template='plotly_white', title='Weekly TCP by Network Length (m)',
           log_x=True, log_y=True)
fig.update_yaxes(title_text='Weekly TCP Mean (m)')
fig.update_xaxes(title_text='Total Network Length (m)')

fig.show()

In [70]:
df[df['region_slug'] == 'montevideo']

,country_name,country_iso,region_slug,region_name,region_type,population,dashboard,timezone,region_shapefile_wkt,rerun,idb_code,waze_code,country_name_idb_eng,region_shapefile_binary,area,n_days,daily_mean,daily_std,daily_coef_var,weekly_mean,weekly_std,weekly_coef_var,daily_approved,weekly_approved,total_length,daily_length_ratio
56,Uruguay,UY,montevideo,Montevideo,city,1363552.0,True,America/Montevideo,Polygon ((-56.23375113878419995 -34.7587515798...,False,UR,UY,Uruguay,b'\x00\x00\x00\x00\x03\x05\x00\x00\x00\xc4H;(\...,0.03,365.0,1.016254e+07,2.652913e+06,0.261048,6.998732e+07,1.380450e+07,0.197243,True,True,3497729.6,2.905468


In [74]:
fig = px.scatter(df.dropna(subset=['weekly_coef_var', 'daily_coef_var'])\
             .query('weekly_coef_var < 1').query('daily_coef_var < 1'),
            y="daily_coef_var", x="total_length", color='region_type',
            hover_name="region_slug", template='plotly_white', title='Daily CV by Network Length (m)',
           log_x=True)
fig.update_yaxes(title_text='Daily CV (m)')
fig.update_xaxes(title_text='Total Network Length (m)')

fig.show()

In [112]:
fig = px.scatter(df,
            y="daily_coef_var", x="daily_osm_ratio", color='region_type',
            hover_name="region_slug", template='plotly_white', title='Daily CV by Network Length (m)',
            log_x=True)
fig.update_yaxes(title_text='Daily CV (m)')
fig.update_xaxes(title_text='Daily TCI Mean over Network Lenth')

fig.write_html(open(OUTPUT_PATH / 'daily-cv-osm.html', 'w'))
fig.show()

In [113]:
fig = px.scatter(df,
            y="weekly_coef_var", x="weekly_osm_ratio", color='region_type',
            hover_name="region_slug", template='plotly_white', title='Weekly CV by Network Length (m)',
            log_x=True)
fig.update_yaxes(title_text='Weekly CV (m)')
fig.update_xaxes(title_text='Weekly TCI Mean over Network Lenth')

fig.show()

ValueError: Value of 'y' is not the name of a column in 'data_frame'. Expected one of ['country_name', 'country_iso', 'region_slug', 'region_name', 'region_type', 'population', 'dashboard', 'timezone', 'region_shapefile_wkt', 'rerun', 'osm_length', 'idb_code', 'waze_code', 'country_name_idb_eng', 'region_shapefile_binary', 'area', 'n_days', 'daily_mean', 'daily_std', 'weekly_mean', 'weekly_std', 'daily_coef_var', 'daily_mean_weekend', 'daily_std_weekend', 'daily_coef_var_weekend', 'daily_mean_weekday', 'daily_std_weekday', 'daily_coef_var_weekday', 'daily_osm_ratio', 'weekend_osm_ratio', 'weekday_osm_ratio', 'weekly_osm_ratio', 'daily_approved', 'weekly_approved'] but received: weekly_coef_var

In [101]:
fig.write_html(open(OUTPUT_PATH / 'weekly-cv-osm.html', 'w'))

In [227]:
from IPython.display import Javascript
from nbconvert import HTMLExporter

def save_notebook():
    display(
        Javascript("IPython.notebook.save_notebook()"),
        include=['application/javascript']
    )

def output_HTML(read_file, output_file):
    import codecs
    import nbformat
    exporter = HTMLExporter()
    # read_file is '.ipynb', output_file is '.html'
    output_notebook = nbformat.read(read_file, as_version=4)
    output, resources = exporter.from_notebook_node(output_notebook)
    codecs.open(output_file, 'w', encoding='utf-8').write(output)
    
def save_now():
    
    import time

    save_notebook()
    time.sleep(3)
    current_file = 'CV and Road Length.ipynb'
    output_file = str(OUTPUT_PATH / 'CV+TCI+ROADLENGTH.html')
    output_HTML(current_file, output_file)